In [1]:
import ezdxf
from  shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import ezdxf.math as math
from collections import Counter
import matplotlib.pyplot as plt
import  pandas as pd
# pandas version 1.1.3
import geopandas
import numpy as np
from shapely.geometry import  Polygon,Point

In [12]:
import time

In [5]:
def  dxf_to_geopandas():
    doc = ezdxf.readfile("D:\VR3D\VR3D_DATASET\J35_txt_cutted\\fichier_2D\BORDEAUX_Travail_J35_ASS.dxf")
    msp = doc.modelspace()
    lwschema = msp.query('LWPOLYLINE')
    polys1 = geopandas.GeoSeries()
    polys2 = geopandas.GeoSeries()
    Rayon = []
    print(len(lwschema))
    for pline in lwschema :
        if not pline.has_arc:
            poly = []
            for p in range(len(pline)):
                x,y,s,e,b = pline[p]
                poly.append((x,y))
            polys1 = polys1.append(geopandas.GeoSeries([Polygon(poly)]),ignore_index = True)
        else :
            polygon_data = {}
            bulg = []
            cx = []
            cy = []
            radius = []
            nxt = 0

            for pl in range(len(pline)):
                x, y, s, e, b = pline[pl]
                bulg.append(b)
                polygon_data[pl] = [x, y]

                if nxt > 0 :
                    C = math.bulge_center(polygon_data[nxt-1],polygon_data[nxt],bulg[nxt-1])
                    cx.append(C[0])
                    cy.append(C[1])
                    R = math.bulge_radius(polygon_data[nxt - 1], polygon_data[nxt], bulg[nxt - 1])
                    radius.append(R)

                if pl+1 == len(pline) :
                    centerX = np.mean(cx)
                    centerY = np.mean(cy)
                    cercleR = np.mean(radius)
                    Rayon.append(cercleR)
                    #AllCercle.append([centerX,centerY,cercleR])
                    cx = []
                    cy = []
                    radius = []
                nxt += 1
            polys1 = polys1.append(geopandas.GeoSeries([Point(centerX,centerY).buffer(cercleR)]),ignore_index = True)
    return polys1
        

In [6]:
poly_geopandas = dxf_to_geopandas()

220


In [7]:
poly_geopandas = poly_geopandas [:219]

In [8]:
def create_fichier_list():
    fichier_list = []
    for i in range(len(poly_geopandas)):
        fichier_list.append('D:\VR3D\VR3D_DATASET\Output_nuage\Out_geopanda\\Nuage_points_'+str(i)+'.txt')
    return  fichier_list

In [23]:
def Tri_nuage_points():
    nuage_data = pd.read_csv("D:\VR3D\VR3D_DATASET\Donner_split\Disk_final\Disk11.txt",header=None,sep = ' ')
    nuage_data = nuage_data.rename(columns={0:'X',1:'Y',2:'Z',3:'I'})
    nuage_data = nuage_data.sort_values(by = ['X','Y'],ignore_index=True)
    return nuage_data,nuage_data[['X','Y']]

In [24]:
nuage_data,couple = Tri_nuage_points()

In [27]:
def train():
    for i in range(50000):
        df = pd.DataFrame(poly_geopandas.contains(Point(couple.loc[i][0],couple.loc[i][1])))
        index = df.index
        condition = df[0] == True
        ind = index[condition]
        if not ind.empty:
            fichier_list = create_fichier_list()
            fichier = open(fichier_list[ind[0]],'a')
            fichier.write(str(couple.loc[i][0])+' '+str(couple.loc[i][1])+' '
                      +str(nuage_data.loc[i][2])+' '+str(nuage_data.loc[i][3])+'\n')
            fichier.close()


In [28]:
%time train()

Wall time: 34.8 s
